In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
insurance = pd.read_csv('Insurance_Data.csv')

In [ ]:
insurance.head()

In [ ]:
insurance.shape

In [ ]:
insurance.drop_duplicates(keep='first', inplace=True)

In [ ]:
insurance.shape ### no duplicated rows

In [ ]:
insurance.columns

In [ ]:
insurance.isnull().sum()

#### No null values, hurray!!!!

In [ ]:
insurance.info()

#### Policy tenure, age of car, and age of policy holder are normalized values.

#### We will seperate max_torque into torque values and max_power into power:

In [ ]:
torque=[]
for i in insurance['max_torque']:
    spl_t=i.split("Nm")
    torque.append(spl_t[0])

In [ ]:
insurance.insert(loc=11, column='torque', value=torque)

In [ ]:
power=[]
for i in insurance['max_power']:
    spl_p=i.split("bhp")
    power.append(spl_p[0])

In [ ]:
insurance.insert(loc=12, column='power', value=power)

In [ ]:
insurance.insert(loc=15, column='volume', value=round((insurance['length']*insurance['width']* insurance['height']),2))

In [ ]:
insurance.drop(['max_power', 'max_torque', 'policy_id', 'length', 'width', 'height'], axis=1, inplace=True)

In [ ]:
insurance.columns

In [ ]:
cat_list=[]
num_list=[]

for i in insurance.columns:
    unique_val=len(insurance[i].unique())
    if i=="area_cluster":
        cat_list.append(i)
    else:
        if unique_val<15:
            cat_list.append(i)
        else:
            num_list.append(i)

### Let's start with the Exploratory Data Analysis  😎 

#### 1. Univariate analysis

In [ ]:
labels = ['0', '1']
size = insurance['is_claim'].value_counts()
colors = ['blue', 'red']

plt.style.use('seaborn-deep')
plt.pie(size, labels=labels, colors = colors, autopct = "%.2f%%")
plt.show()

#### Highly imbalanced dataset, we need to do some upsampling (will be done later)  🔨 

In [ ]:
def dist_box(data):
    Name=data.name.upper()
    fig,(ax_box,ax_dis)  =plt.subplots(2,1,gridspec_kw = {"height_ratios": (.25, .75)},figsize=(8, 5))
    mean=data.mean()
    median=data.median()
    mode=data.mode().tolist()[0]
    fig.suptitle("SPREAD OF DATA FOR "+ Name  , fontsize=18, fontweight='bold')
    sns.boxplot(x=data,showmeans=True, orient='h',color="violet",ax=ax_box)
    ax_box.set(xlabel='')
    sns.distplot(data,kde=False,color='blue',ax=ax_dis)
    ax_dis.axvline(mean, color='r', linestyle='--',linewidth=2)
    ax_dis.axvline(median, color='g', linestyle='-',linewidth=2)
    ax_dis.axvline(mode, color='y', linestyle='-',linewidth=2)
    plt.legend({'Mean':mean,'Median':median,'Mode':mode})

for i in range(len(num_list)):
    dist_box(insurance[num_list[i]])


#### Observations  🧐 
    ####1. Max are new policies
    ####2. People tend to buy policies for their new cars, mostly
    ####3. After 0.5 of age, less people tend to buy policies.
    ####4. Population_density, although seem to be numerical, is actually categorial!!

In [ ]:
for col in cat_list:
    plt.figure(figsize=(18,15))
    sns.countplot(x=col, data=insurance, palette='Set2')
    plt.show()
    print("no. of unique values of", col, "is: ", insurance[col].nunique())
    print("**************************************************************")
    
    

#### Observations:
    #### 1. High number of categories>10 exist in area_cluster, model, engine_type, height
    #### 2. C8>C2>C5 clusters
    #### 3. Make 1 dominates the market
    #### 4. B2 and A segments are the highest, utility is significantly low
    #### 5. Models M1, M4 and M6 constitute a significant portion of models in market.
    #### 6. Petrol and CNG fuel types the most common.
    #### 7. F&D Petrol, 1.5 L U2 CRD, and K series Dual jet engines are high
    #### 8. Cars with 2 airbags are most common.
    #### 9. Majority cars have no TPMS, no electronic stability control (ESC) system, no parking camers, no rear window wiper, drum rear brakes, manual transmission, 5 gear boxes, power steering, front fog lights, power door locks, central locking sytem, speed alert
    #### 10. NCAP rating of 2 is the most common, while NCAP rating of 0 is also quite high. Why??? ☹️

In [ ]:
insurance.groupby(['fuel_type'])['fuel_type'].count().plot.pie(figsize=(7,7))
plt.title("Proportion of fuel types")
plt.show()

In [ ]:
insurance.groupby(['airbags'])['airbags'].count().plot.pie(figsize=(7,7))
plt.title("Proportion of airbags")
plt.show()

In [ ]:
insurance.groupby(['rear_brakes_type'])['rear_brakes_type'].count().plot.pie(figsize=(7,7))
plt.title("Proportion of rear brake types")
plt.show()

#### 2. Bivariate analysis

In [ ]:
plt.figure(figsize=(5,5))
sns.histplot(
    insurance, x="age_of_car",  hue="is_claim", palette="Set1" , bins=25, element="step",  common_norm=False,
    stat="probability"
)
plt.show()

#### Almost same distribution of car age with the target variable

In [ ]:
plt.figure(figsize=(5,5))

sns.histplot(
    insurance, x="policy_tenure",  hue="is_claim", palette="Set2" , bins=50, element="step",  common_norm=False,
    stat="probability"
)
plt.show()

#### Higher policy tenure leads to higher probability of claim

In [ ]:
plt.figure(figsize=(5,5))
sns.histplot(
    insurance, x="age_of_policyholder",  hue="is_claim", palette="Set2" , bins=50, element="step",  common_norm=False,
    stat="probability"
)
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='make', hue ='is_claim', data=insurance, palette='Set2')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='ncap_rating', hue ='is_claim', data=insurance, palette='Set1')
plt.show()

#### No claims passed for the safest cars!!

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='is_ecw', hue ='is_claim', data=insurance, palette='Set2')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='is_parking_sensors', hue ='is_claim', data=insurance, palette='Set2')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='is_central_locking', hue ='is_claim', data=insurance, palette='Set2')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='is_power_door_locks', hue ='is_claim', data=insurance, palette='Set2')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='steering_type', hue ='is_claim', data=insurance, palette='Set2')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='ncap_rating', hue ='transmission_type', data=insurance)
plt.show()

#### ⭐⭐⭐ rating cars are fully automatic

In [ ]:
sns.scatterplot(x ='age_of_car',y ='age_of_policyholder', hue="fuel_type", data = insurance)
plt.show()

#### Outlier treatment

In [ ]:
for i in ['policy_tenure', 'age_of_car', 'age_of_policyholder']:
    percentile25 = insurance[i].quantile(0.25)
    percentile75 = insurance[i].quantile(0.75)

    iqr = percentile75 - percentile25
    
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr
    oulier_count=len(insurance[insurance[i]>upper_limit])+len(insurance[insurance[i]<lower_limit])
    print("The no. of outliers in", i, 'is', oulier_count)
    upper_array = np.where(insurance[i]>=upper_limit)[0]
    lower_array = np.where(insurance[i]<=lower_limit)[0]
 
    # Removing the outliers
    insurance.drop(index=upper_array, inplace=True)
    insurance.drop(index=lower_array, inplace=True)

In [ ]:
insurance.shape

#### Determining numerical, ordinal, and nominal cols

In [ ]:
num_cols=['policy_tenure', 'age_of_car', 'age_of_policyholder']
ord_cols=['population_density', 'torque', 'power', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 
          'volume', 'gross_weight', 'ncap_rating']


In [ ]:
nom_cols=[]
for i in insurance.columns[:40]:
    if i not in ord_cols and i not in num_cols:
        nom_cols.append(i)

In [ ]:
(nom_cols)

#### Encoding both nominal and ordincal categories

In [ ]:
X = insurance.drop(['is_claim'], axis=1)
y = insurance["is_claim"]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()

In [ ]:
for col in ord_cols:
    x_train[col]=oe.fit_transform(x_train[[col]])

In [ ]:
x_train_enc = pd.get_dummies(x_train, columns = nom_cols, drop_first= True)

In [ ]:
x_train_enc.head()

In [ ]:
for col in ord_cols:
    x_test[col]=oe.fit_transform(x_test[[col]])

In [ ]:
x_test_enc = pd.get_dummies(x_test, columns = nom_cols, drop_first= True)

In [ ]:
x_test_enc.head()

### Feature selection
#### 1. Correlation with target

In [ ]:
x_all=pd.concat([x_train_enc, y_train], axis=1) ### Checking for low correlation with target!!
cor=x_all.corr()["is_claim"].sort_values(ascending=False)
correlate=pd.DataFrame({"Column":cor.index,"Correlation with target":cor.values})

In [ ]:
correlate.head(20)

In [ ]:
low_corr=correlate.loc[abs(correlate['Correlation with target']) <= 0.005, 'Column'].values

In [ ]:
len(low_corr) ## Found 38 low-correlation columns

In [ ]:
x_all_1=x_all.drop(columns=low_corr)
corr= x_all_1.corr()
# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(corr)

# using the upper triangle matrix as mask 
plt.figure(figsize=(40,40))
plt.title("Heatmap excluding the low-correlation columns", fontsize=20)
sns.heatmap(corr, annot=True, mask=matrix, cmap="coolwarm")
plt.show()

In [ ]:
### Some more columns need to be removed!!!

#### 2. Multicollinearity of independent variables

In [ ]:
columns = x_all_1.corr().columns

# Create an empty list to keep track of columns to drop
columns_to_drop = []

# Loop over the columns
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        # Access the cell of the DataFrame
        if (x_all_1.corr().loc[columns[i], columns[j]]) > 0.8 or (x_all_1.corr().loc[columns[i], columns[j]]) <-0.8:
            columns_to_drop.append(columns[j])

print(len(columns_to_drop))

In [ ]:
columns_to_drop = set(columns_to_drop) ## using set as there are duplicate columns

In [ ]:
len(columns_to_drop)

In [ ]:
x_all_2=x_all_1.drop(columns=columns_to_drop)
corr=x_all_2.corr()
# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(corr)

# using the upper triangle matrix as mask 
plt.figure(figsize=(28,28))
plt.title("Heatmap excluding the low-correlation and multicollinearity", fontsize=20)
sns.heatmap(corr, annot=True, mask=matrix, cmap="coolwarm")
plt.show()

In [ ]:
len(x_all_2.columns)

#### Modelling starts!! We will be using different cases, shown below

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

#### Case 1: Using dataset without samling or FS (feature selection)

In [ ]:
lr=LogisticRegression(random_state=42)
dtc=DecisionTreeClassifier(random_state=42)
rfc=RandomForestClassifier(random_state=42)
gbc=GradientBoostingClassifier(random_state=42)
xgb=XGBClassifier(random_state=42)
ada = AdaBoostClassifier(random_state=42)

In [ ]:
models=[lr,dtc,rfc,gbc, xgb, ada]

In [ ]:
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold ## better for dealing with imbalanced datasets
for model in models:
    print("Model: ", model)
    for scoring in["balanced_accuracy", "f1"]:
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
        scores = cross_val_score(model, x_train_enc, y_train, scoring=scoring, cv=cv, n_jobs=-1)
        
        print(scoring, " mean=", scores.mean(), " SD= ", scores.std())
        
    print("*********************")

In [ ]:
def model_evaluation(models, x_train, y_train, x_test):
    for model in models:
    
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        accuracy = round(balanced_accuracy_score(y_test, y_pred),3)
        precision = round(precision_score(y_test, y_pred), 3)
        recall = round(recall_score(y_test, y_pred), 3)
        F1_score = round(f1_score(y_test, y_pred), 3)
    
        print("For ", model)
        print("Balanced accuracy: ", accuracy, " Precision: ", precision, " Recall: ", recall, " F1 score: ", F1_score)
    
        
        classif_rep = classification_report(y_test, y_pred)
        print("Classification Report:\n", classif_rep)
        


In [ ]:
print("CASE 1: UNTREATED DATASET:      \n")
model_evaluation(models, x_train_enc, y_train, x_test_enc)

#### Case 2: Using upsampled data

In [ ]:
from imblearn.over_sampling import RandomOverSampler
sampler = RandomOverSampler(random_state=1)
x_train_over, y_train_over = sampler.fit_resample(x_train_enc,y_train)

In [ ]:
print("Before resampling: ", y_train.value_counts())
print("After resampling: ", y_train_over.value_counts())

In [ ]:
print("CASE 2: UPSAMPLED DATASET:      \n")
model_evaluation(models, x_train_over, y_train_over, x_test_enc)

#### Case 3: Using FS (low-correlation and multicollinearity excluded)

In [ ]:
x_train_sel=x_train_over.drop(columns=low_corr)
x_test_sel=x_test_enc.drop(columns=low_corr)

In [ ]:
x_train_sel=x_train_sel.drop(columns=columns_to_drop)
x_test_sel=x_test_sel.drop(columns=columns_to_drop)

In [ ]:
x_train_sel.shape[1]

In [ ]:
print("CASE 3: FEATURE SELECTION USING CORRELATION:      \n")
model_evaluation(models, x_train_sel, y_train_over, x_test_sel)

#### Case 4: Using FS (chi square) method

In [ ]:
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

# Calculate chi-squared stats
chi_scores = chi2(x_train_over, y_train_over)

# chi_scores[1] are the p-values of each feature.
p_values = pd.Series(chi_scores[1], index = x_train_over.columns)
p_values.sort_values(inplace = True)

# Plotting the p-values
plt.figure(dpi=100, figsize=(20, 20))
p_values.plot.bar()

plt.title('Chi-square test - P-values', fontsize=18)
plt.xlabel('Feature')
plt.ylabel('P-value')

plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest
sel_30_cols = SelectKBest(score_func=chi2, k=30) ## selecting 30 best columns
sel_30_cols.fit(x_train_over, y_train_over)
sel_cols=x_train_over.columns[sel_30_cols.get_support()]

In [ ]:
x_train_chisquare = x_train_over.loc[:,x_train_over.columns.isin(sel_cols)]

In [ ]:
x_test_chisquare = x_test_enc.loc[:,x_test_enc.columns.isin(sel_cols)]

In [ ]:
print("CASE 4: FEATURE SELECTION USING CHI SQUARE TEST      \n")
model_evaluation(models, x_train_chisquare, y_train_over, x_test_chisquare)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sampler_under = RandomUnderSampler(random_state=1)
x_train_under, y_train_under = sampler_under.fit_resample(x_train_enc,y_train)

In [ ]:
print("CASE 5: DOWNSAMPLED DATASET      \n")
model_evaluation(models, x_train_under, y_train_under, x_test_enc)

#### Comparing all the cases, we see that GradBoost Classifier results in the highest accuracy and F1 score, when trained on the upsampled data with feature selection (case 3). 

#### So now, I will perform a hyperparameter tuning!!

### Hyperparameter tuning (Warning: Very computationally expensive!)

In [ ]:
from sklearn.model_selection import GridSearchCV
p_test1 = {'learning_rate':[0.01,0.05,0.1,0.15,0.2], 'n_estimators':[100,250,500,750,1000,1250,1500]}

tuning = GridSearchCV(estimator =GradientBoostingClassifier(max_depth=6, min_samples_split=200, min_samples_leaf=40, subsample=1,max_features='sqrt', random_state=10), param_grid = p_test1, scoring='f1',n_jobs=4, cv=5)
tuning.fit(x_train_sel, y_train_over)

tuning.best_params_, tuning.best_score_

In [ ]:
tuning.best_params_

In [ ]:
p_test2 = {'max_depth':[3,4,5,6,7,8] }

tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=0.2,n_estimators=1500,min_samples_split=200, min_samples_leaf=40, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = p_test2, scoring='f1',n_jobs=4, cv=5)
tuning.fit(x_train_sel, y_train_over)

tuning.best_params_, tuning.best_score_


##### Below, I have manually tuned the hyperparameters again to find the optimum values
##### learning_rate=0.02, n_estimators=300,max_depth=4, min_samples_split=700, min_samples_leaf=30,max_features=5.

In [ ]:
new_model=GradientBoostingClassifier(learning_rate=0.02, n_estimators=300,max_depth=4, min_samples_split=700, min_samples_leaf=30,max_features=5, subsample=1, random_state=10)
new_model.fit(x_train_sel, y_train_over)
y_pred=new_model.predict(x_test_sel)
print("Final balanced accuracy", round(balanced_accuracy_score(y_test, y_pred),3))
print("Final F1 score", round(f1_nscore(y_test, y_pred), 3))

#### Not any improvement seen !!

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve 

In [ ]:
model_roc_auc = roc_auc_score(y_test, y_pred)
fpr1, tpr1, thresholds1 = roc_curve(y_test, new_model.predict_proba(x_test_sel)[:,1])

In [ ]:
plt.figure()
plt.plot(fpr1, tpr1, label='XBG Model (area = %0.2f)' % model_roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.show()